## Load data

Data is generated by a RaspberryPi with a temperature and humidity sensor at home. Readings are taken every 10 minutes using a bash script, then stored onto DynamoDB. However there have been some period of missing data, e.g. when I got a new WiFi network and forgot to update the password on the Pi. This notebook loads and formats the data, then replaces missing periods with the previous day. 

In [73]:
import numpy as np
import pandas as pd
import plotly.express as px
from datetime import timedelta

In [79]:
df = pd.read_csv('ddb_output.csv'
         ).rename(columns={'humidity.S': 'humidity',
                   'temperature.S':'temperature',
                   'timestamp.S':'timestamp'},
         ).drop(columns=['Unnamed: 0']
         ).sort_values(by='timestamp')
# Convert the timestamp column to datetime format
df['timestamp'] = pd.to_datetime(df['timestamp'], format='mixed')

# Round the timestamp to the nearest minute
df['timestamp'] = df['timestamp'].dt.round('1min')

print(df.shape)

(16134, 3)


Missing periods are visible below. 

In [76]:
fig = px.scatter(df, x="timestamp", y=["humidity","temperature"], title='All-time humidity and temperature in the grove!')
fig.show()

In [29]:
df_original = df

Identify missing entries, and create rows with N/A for them

In [80]:
time_interval = timedelta(minutes=10)

i = 0 

while i < df.shape[0]-1: 

    current_time = pd.Timestamp(df.iloc[i]['timestamp'])
    next_time = pd.Timestamp(df.iloc[i + 1]['timestamp'])
    
    # Check if the time interval between current and next timestamp is much longer than we expect
    if (next_time - current_time) > time_interval + timedelta(minutes=25):
        
        # Insert a new row with NA values and a timestamp 10 minutes after the current timestamp
        new_row = pd.DataFrame({'timestamp': [current_time + time_interval],
                                'humidity': [np.nan],
                                'temperature':[np.nan]})
        
        # Concatenate the new row to the dataframe
        df = pd.concat([df.iloc[:i+1], new_row, df.iloc[i+1:]], ignore_index=True)

    i += 1

# Updated dataframe with inserted rows for missing timestamps
print(df.shape)

(17301, 3)


What proportion of the data is missing?

In [92]:
rows_missing = (df.shape[0]-df_original.shape[0])/df.shape[0]

print(f'We identified {np.round(rows_missing*100,2)}% of rows as missing, '
      f'out of an enriched datamframe with {df.shape[0]} rows.')

We identified 6.75% of rows as missing, out of an enriched datamframe with 17301 rows.


Let's fill using the previous day's readings (thanks to my partner for this idea!)


In [94]:
values_changed = 0

one_day = 10 * 24 # Number of readings ago

# Loop through the dataframe
for i in range(one_day, len(df)):
    current_value = df.loc[i, 'temperature']
    
    # Check if the value is NaN
    if pd.isna(current_value):

        # Set the value equal to the value from 240 rows ago
        previous_value_temp = df.loc[i-one_day, 'temperature']
        df.loc[i, 'temperature'] = previous_value_temp

        # Set the value equal to the value from 240 rows ago
        previous_value_humidity = df.loc[i-one_day, 'humidity']
        df.loc[i, 'humidity'] = previous_value_humidity

In [95]:
df.shape

(17301, 3)

In [96]:
fig = px.scatter(df, x="timestamp", y=["humidity","temperature"], title='All-time humidity and temperature in the grove!')
fig.show()

This does lead to some strange discontinous patterns, but overall seems close enough to reality. 

In [98]:
df.shape

(17301, 3)

In [97]:
df[:600]

# Took until midday on 30th April to stabilise on one reading every ten mins

,humidity,temperature,timestamp
0,56.0,17.83,2023-04-28 08:25:00
1,56.0,17.85,2023-04-28 08:26:00
2,56.0,17.85,2023-04-28 08:28:00
3,56.0,17.88,2023-04-28 08:32:00
4,NaN,NaN,2023-04-28 08:42:00
...,...,...,...
595,55.0,20.64,2023-04-30 17:50:00
596,55.0,20.59,2023-04-30 18:00:00
597,55.0,20.70,2023-04-30 18:10:00
598,55.0,20.58,2023-04-30 18:20:00


Selecting only data past that point, so data is evenly spaced.

In [99]:
df = df[df['timestamp'] > pd.Timestamp(year=2023, month=5, day=1)]

In [102]:
df.to_csv('analysis/historic_readings_no_missing_data.csv')